## KNN con distintos tipos de codificacion de las variables categoricas

    -15 vecinos
    -Por ahora dejo los valores default(weight = uniform, p = 2, leaf_size = 30)
    -SimpleImputer(Mean)
    Codificacion:
        -Label encoding
        -Binary encoding
        -One hot encoding
        -Target encoding ? (Preguntar si tiene sentido)

In [2]:
import numpy as np
import pandas as pd
from common import metrica
pd.set_option('mode.chained_assignment', None) # Deshabilita SettingWithCopyWarning. Ojo.

In [3]:
df = pd.read_csv('sets_de_datos/train.csv')

In [4]:
df["fecha"] = pd.to_datetime(df["fecha"])

In [5]:
from sklearn.model_selection import train_test_split
from category_encoders import BinaryEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

In [16]:
from sklearn.neighbors import KNeighborsRegressor

In [25]:
cant_vecinos = [15, 30, 50, 100]

In [7]:
col_borrar = ["direccion", "lat", "lng", "titulo", "descripcion", "idzona", "fecha"]
df = df.drop(col_borrar, axis=1)

In [8]:
col_categorias = ["ciudad", "provincia", "tipodepropiedad"]
df_categorico = df[col_categorias].dropna()

In [9]:
col_datos = list(set(df.columns.tolist()) - set(col_categorias))

In [10]:
col_a_imputar = list(set(df.columns[df.isnull().sum() > 0].tolist()) - set(col_categorias))

## ============================Binary Encoder============================= 

In [11]:
codificador_bin = BinaryEncoder(cols=col_categorias)
df_binario = codificador_bin.fit_transform(df_categorico)
df_binario = df_binario.join(df[col_datos], how="inner")
df_binario

,ciudad_0,ciudad_1,ciudad_2,ciudad_3,ciudad_4,ciudad_5,ciudad_6,ciudad_7,ciudad_8,ciudad_9,...,id,precio,habitaciones,banos,metrostotales,antiguedad,escuelascercanas,gimnasio,metroscubiertos,garages
0,0,0,0,0,0,0,0,0,0,0,...,254099,2273000.0,2.0,2.0,80.0,NaN,0.0,0.0,80.0,1.0
1,0,0,0,0,0,0,0,0,0,1,...,53461,3600000.0,3.0,2.0,180.0,10.0,1.0,0.0,268.0,2.0
2,0,0,0,0,0,0,0,0,0,1,...,247984,1200000.0,3.0,2.0,166.0,5.0,0.0,0.0,144.0,2.0
3,0,0,0,0,0,0,0,0,1,0,...,209067,650000.0,2.0,1.0,67.0,1.0,1.0,0.0,63.0,1.0
4,0,0,0,0,0,0,0,0,1,0,...,185997,1150000.0,2.0,1.0,95.0,10.0,0.0,0.0,95.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,0,0,0,0,0,0,0,0,1,0,...,119879,650000.0,2.0,1.0,NaN,0.0,0.0,0.0,67.0,2.0
239996,0,0,0,0,1,0,1,0,0,0,...,259178,1940000.0,3.0,3.0,250.0,0.0,1.0,0.0,200.0,3.0
239997,0,0,0,0,0,0,0,0,0,0,...,131932,3400000.0,2.0,2.0,138.0,20.0,0.0,0.0,138.0,1.0
239998,0,0,0,0,1,0,0,0,0,0,...,146867,2890000.0,4.0,4.0,137.0,20.0,1.0,1.0,235.0,0.0


In [12]:
datos_bin = df_binario.drop(["precio"], axis=1)
precios_bin = df_binario["precio"]
datos_train_bin, datos_test_bin, precio_train_bin, precio_test_bin = train_test_split(datos_bin, precios_bin, test_size=0.25, random_state=42)

In [13]:
for c in col_a_imputar:
    imp = SimpleImputer(strategy="mean")
    datos_train_bin[c] = imp.fit_transform(datos_train_bin[[c]])
    datos_test_bin[c] = imp.fit_transform(datos_test_bin[[c]])

In [26]:
pred_bin = []
for cant in cant_vecinos:
    vecinos_bin = KNeighborsRegressor(n_neighbors=cant, algorithm="kd_tree").fit(datos_train_bin, precio_train_bin)
    pred_bin.append(vecinos_bin.predict(datos_test_bin))

In [27]:
for n in range(len(pred_bin)):
    print("#Vecinos: ", cant_vecinos[n])
    metrica.resultados(precio_test_bin, pd.Series(pred_bin[n]))


#Vecinos:  15
+-------------------------+
| RMSLE   | MAE           |
+-------------------------+
| 0.99131 | 1816473.57994 |
+-------------------------+
#Vecinos:  30
+-------------------------+
| RMSLE   | MAE           |
+-------------------------+
| 0.96782 | 1768784.67382 |
+-------------------------+
#Vecinos:  50
+-------------------------+
| RMSLE   | MAE           |
+-------------------------+
| 0.94918 | 1731087.33442 |
+-------------------------+
#Vecinos:  100
+-------------------------+
| RMSLE   | MAE           |
+-------------------------+
| 0.92001 | 1668851.98998 |
+-------------------------+


## =========================Label Encoder================================

In [31]:
codificador_le = LabelEncoder()
df_le = df_categorico.apply(codificador_le.fit_transform)
df_le = df_le.join(df[col_datos], how="inner")
df_le

,ciudad,provincia,tipodepropiedad,piscina,usosmultiples,centroscomercialescercanos,id,precio,habitaciones,banos,metrostotales,antiguedad,escuelascercanas,gimnasio,metroscubiertos,garages
0,93,8,0,0.0,0.0,0.0,254099,2273000.0,2.0,2.0,80.0,NaN,0.0,0.0,80.0,1.0
1,370,8,3,0.0,0.0,1.0,53461,3600000.0,3.0,2.0,180.0,10.0,1.0,0.0,268.0,2.0
2,767,14,2,0.0,0.0,0.0,247984,1200000.0,3.0,2.0,166.0,5.0,0.0,0.0,144.0,2.0
3,861,10,2,0.0,0.0,1.0,209067,650000.0,2.0,1.0,67.0,1.0,1.0,0.0,63.0,1.0
4,851,14,0,0.0,0.0,0.0,185997,1150000.0,2.0,1.0,95.0,10.0,0.0,0.0,95.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,861,10,2,0.0,0.0,0.0,119879,650000.0,2.0,1.0,NaN,0.0,0.0,0.0,67.0,2.0
239996,766,10,2,0.0,0.0,1.0,259178,1940000.0,3.0,3.0,250.0,0.0,1.0,0.0,200.0,3.0
239997,93,8,0,0.0,0.0,0.0,131932,3400000.0,2.0,2.0,138.0,20.0,0.0,0.0,138.0,1.0
239998,333,8,2,0.0,0.0,1.0,146867,2890000.0,4.0,4.0,137.0,20.0,1.0,1.0,235.0,0.0


In [32]:
datos_le = df_le.drop(["precio"], axis=1)
precios_le = df_le["precio"]
datos_train_le, datos_test_le, precio_train_le, precio_test_le = train_test_split(datos_le, precios_le, test_size=0.25, random_state=42)

In [33]:
for c in col_a_imputar:
    imp = SimpleImputer(strategy="mean")
    datos_train_le[c] = imp.fit_transform(datos_train_le[[c]])
    datos_test_le[c] = imp.fit_transform(datos_test_le[[c]])

In [34]:
pred_le = []
for cant in cant_vecinos:
    vecinos_le = KNeighborsRegressor(n_neighbors=cant, algorithm="kd_tree").fit(datos_train_le, precio_train_le)
    pred_le.append(vecinos_le.predict(datos_test_le))

In [35]:
for n in range(len(pred_le)):
    print("#Vecinos: ", cant_vecinos[n])
    metrica.resultados(precio_test_le, pd.Series(pred_le[n]))


#Vecinos:  15
+-------------------------+
| RMSLE   | MAE           |
+-------------------------+
| 0.96465 | 1755576.98932 |
+-------------------------+
#Vecinos:  30
+-------------------------+
| RMSLE   | MAE           |
+-------------------------+
| 0.93548 | 1701194.22019 |
+-------------------------+
#Vecinos:  50
+-------------------------+
| RMSLE   | MAE           |
+-------------------------+
| 0.91675 | 1661777.98031 |
+-------------------------+
#Vecinos:  100
+-------------------------+
| RMSLE   | MAE           |
+-------------------------+
| 0.90013 | 1620996.31193 |
+-------------------------+


## ==========================One Hot Encoding==========================

In [36]:
df_ohe = pd.get_dummies(df, dummy_na=True)
df_ohe

,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,gimnasio,usosmultiples,piscina,...,provincia_San luis Potosí,provincia_Sinaloa,provincia_Sonora,provincia_Tabasco,provincia_Tamaulipas,provincia_Tlaxcala,provincia_Veracruz,provincia_Yucatán,provincia_Zacatecas,provincia_nan
0,254099,NaN,2.0,1.0,2.0,80.0,80.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,53461,10.0,3.0,2.0,2.0,268.0,180.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,247984,5.0,3.0,2.0,2.0,144.0,166.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,209067,1.0,2.0,1.0,1.0,63.0,67.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,185997,10.0,2.0,1.0,1.0,95.0,95.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,119879,0.0,2.0,2.0,1.0,67.0,NaN,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
239996,259178,0.0,3.0,3.0,3.0,200.0,250.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
239997,131932,20.0,2.0,1.0,2.0,138.0,138.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
239998,146867,20.0,4.0,0.0,4.0,235.0,137.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
datos_ohe = df_ohe.drop(["precio"], axis=1)
precios_ohe = df_ohe["precio"]
datos_train_ohe, datos_test_ohe, precio_train_ohe, precio_test_ohe = train_test_split(datos_ohe, precios_ohe, test_size=0.25, random_state=42)

In [39]:
for c in col_a_imputar:
    imp = SimpleImputer(strategy="mean")
    datos_train_ohe[c] = imp.fit_transform(datos_train_ohe[[c]])
    datos_test_ohe[c] = imp.fit_transform(datos_test_ohe[[c]])

In [ ]:
pred_ohe = []
for cant in cant_vecinos:
    vecinos_ohe = KNeighborsRegressor(n_neighbors=15, algorithm="kd_tree").fit(datos_train_ohe, precio_train_ohe)
    pred_ohe.append(vecinos_ohe.predict(datos_test_ohe))

In [1]:
#Demasiado para mi pobre compu

## ===========================Target Encoding===========================